In [1]:
# Getting the dataset of 15 family of mushroom
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset
import os

In [1]:
path = "C:/Users/mbrei/OneDrive/Bureau/ChampIA/Data/data_image"

We create a class to get the path of each image according to family and data type to transform it in tensors


In [9]:
class MushroomDataset(Dataset):
        def __init__(self, root_dir, data_type, transform=None):
                """
                Choisir un répertoire racine et une transformation à appliquer aux images
                """
                self.root_dir = root_dir
                self.data_type = data_type 
                self.transform = transform
                self.data = []
                self.classes = {}
                self._load_images()
                
        def _load_images(self):
                """
                Récupérer les chemins de chaque image via différents dossiers avec l'étiquette de la famille de champignons
                """
                class_idx = 0
                for family_name in os.listdir(self.root_dir):
                        family_dir = os.path.join(self.root_dir, family_name)
                        if os.path.isdir(family_dir):
                                data_type_dir = os.path.join(family_dir, 'data', self.data_type, family_name).replace('\\', '/')
                                data_type_dir_champ = os.path.join(family_dir, 'data', self.data_type, f"mushrooms {family_name}").replace('\\', '/')
                                if os.path.exists(data_type_dir):
                                        target_dir = data_type_dir
                                elif os.path.exists(data_type_dir_champ):
                                        target_dir = data_type_dir_champ
                                else:
                                        continue
                                if family_name not in self.classes:
                                        self.classes[family_name] = class_idx
                                        class_idx += 1
                                for img_file in os.listdir(target_dir):
                                        img_path = os.path.join(target_dir, img_file).replace('\\', '/')
                                        if img_file.lower().endswith(('.jpg')):
                                                self.data.append((img_path, family_name))
        
        def __len__(self):
                return len(self.data)
        
        def __getitem__(self, idx):
                img_path, family_name = self.data[idx]
                image = Image.open(img_path).convert("RGB")
                
                if self.transform:
                        image = self.transform(image)
                
                # Convertir le nom de famille en index numérique
                label = self.classes[family_name]
                
                return image, label

In [10]:
# Data with only train
transform = transforms.Compose([transforms.ToTensor()])

train_data = MushroomDataset(root_dir=path, data_type='train', transform=transform)

In [13]:
train_data[0]

(tensor([[[0.8471, 0.7647, 0.7608,  ..., 0.4745, 0.3922, 0.3176],
          [0.7843, 0.7098, 0.7765,  ..., 0.3529, 0.3333, 0.2353],
          [0.7765, 0.7294, 0.6471,  ..., 0.4824, 0.4275, 0.3765],
          ...,
          [0.5294, 0.5647, 0.3843,  ..., 0.5294, 0.3686, 0.2275],
          [0.3373, 0.3216, 0.3804,  ..., 0.1725, 0.1451, 0.4275],
          [0.3020, 0.2157, 0.1216,  ..., 0.3294, 0.4706, 0.5059]],
 
         [[0.8353, 0.7176, 0.6706,  ..., 0.4078, 0.3686, 0.3216],
          [0.7569, 0.6549, 0.6784,  ..., 0.2588, 0.2667, 0.1843],
          [0.7294, 0.6549, 0.5451,  ..., 0.3569, 0.3059, 0.2510],
          ...,
          [0.3373, 0.3765, 0.1882,  ..., 0.4784, 0.2667, 0.0706],
          [0.1725, 0.1569, 0.2196,  ..., 0.1216, 0.0667, 0.3216],
          [0.1569, 0.0784, 0.0078,  ..., 0.2549, 0.4196, 0.4824]],
 
         [[0.6745, 0.6314, 0.6392,  ..., 0.3294, 0.3137, 0.2902],
          [0.6471, 0.6039, 0.6627,  ..., 0.2510, 0.2902, 0.2235],
          [0.6824, 0.6392, 0.5490,  ...,

In [14]:
# Transformation des images pour les amener à la bonne forme et les normaliser
transform = transforms.Compose([
  transforms.Resize((128, 128)),  # Assurer que l'image est bien de taille 128x128\n",
  transforms.ToTensor(),  # Convertir l'image en un tenseur\n",
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisation\n",
  ]),
# Supposons que train_data est un objet Dataset personnalisé, donc on l'enveloppe dans un DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [15]:
import torch.nn as nn
import torch.optim as optim
class SimpleCNN(nn.Module):

        def __init__(self, num_classes):
            super(SimpleCNN, self).__init__()
            
            # Définir les couches du CNN
            self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
            self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
            self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
            
            # MaxPooling pour réduire la taille de l'image
            self.pool = nn.MaxPool2d(2, 2)
            
            # Couches Fully Connected
            self.fc1 = nn.Linear(64 * 16 * 16, 512)  # Assurez-vous que les dimensions correspondent
            self.fc2 = nn.Linear(512, num_classes)  # Nombre de classes
            
        def forward(self, x):
            # Appliquer les convolutions et les max pooling
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            x = self.pool(torch.relu(self.conv3(x)))
            # Applatir l'image pour la passer dans les couches fully connected
            x = x.view(-1, 64 * 16 * 16)
            
            # Appliquer les couches fully connected
            x = torch.relu(self.fc1(x))
            x = self.fc2(x)
            
            return x

In [16]:
# Initialiser le modèle avec le nombre de classes
num_classes = len(train_data.classes)
model = SimpleCNN(num_classes=num_classes)
# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Définir l'appareil (GPU ou CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Entraîner le modèle
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Mode entraînement
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        # Passer les entrées dans le modèle
        outputs = model(inputs)
        
        # Calculer la perte
        loss = criterion(outputs, labels)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        # Calculer la précision
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_loss += loss.item()
    epoch_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    
    print(f"Époque {epoch+1}/{num_epochs}, Perte : {epoch_loss:.4f}, Précision : {accuracy:.2f}%")
   

Époque 1/10, Perte : 2.1998, Précision : 24.73%
Époque 2/10, Perte : 1.8576, Précision : 37.92%
Époque 3/10, Perte : 1.6185, Précision : 46.17%
Époque 4/10, Perte : 1.3321, Précision : 56.04%
Époque 5/10, Perte : 0.9745, Précision : 68.17%
Époque 6/10, Perte : 0.5321, Précision : 82.83%
Époque 7/10, Perte : 0.2089, Précision : 93.49%
Époque 8/10, Perte : 0.1066, Précision : 97.21%
Époque 9/10, Perte : 0.0511, Précision : 98.58%
Époque 10/10, Perte : 0.0379, Précision : 98.98%


In [17]:
torch.save(model.state_dict(), "mushroom_cnn.pth")